In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from arcgis.features import GeoAccessor

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

import getpass # library to get dynamically input AGOL Password

In [ ]:
class ArcGISDataAnalyzer:
    def __init__(self, target_date):
        self.password = getpass.getpass("Please enter your ArcGIS Online password: ") #Prompt the user to enter ArcGIS Online Password # password parameter to prompt user for ArcGIS Online password
        self.gis = GIS("<replace with your ArcGIS Online organization URL>", "<replace with your ArcGIS Online username>", self.password) # Connect to ArcGIS Online using the specified username and password
        self.target_date = datetime.strptime(target_date, "%Y-%m-%d").date()
        self.items_before_date_all_users_df = None
        self.sorted_user_details = None
        
        self.sorted_user_details = None
        
    # Define a function to calculate credits per month based on item type and size
    def _calculate_credits_per_month(self, item_type, item_size):
        if item_type == 'Feature Service':
            credits_per_10mb = 2.4
            credits_per_month = (item_size / 10) * credits_per_10mb
        elif item_type == 'Tiled Imagery':
            credits_per_gb = 1.2
            credits_per_month = (item_size / 1024) * credits_per_gb
        elif item_type == 'Dynamic Imagery':
            num_images = 100  # Example number of images
            if num_images <= 10:
                credits_per_day = 10
            elif num_images <= 100:
                credits_per_day = 20
            elif num_images <= 1000:
                credits_per_day = 40
            elif num_images <= 10000:
                credits_per_day = 80
            elif num_images <= 100000:
                credits_per_day = 160
            else:
                credits_per_day = 320
            credits_per_month = (credits_per_day * 30)  # Assuming 30 days in a month
        else:
            credits_per_month = (item_size / 10) * 2.4  # Default value if item type is not found
        return credits_per_month

    def get_items_before_date_all_users(self):
        
        # Retrieve all users in the organization
        all_users = self.gis.users.search(query="*")  # You can modify the query as needed

        # Create a list to store item information
        all_items = []

        # Iterate through each user and retrieve their items
        for user in all_users:
            user_items = user.items()
            for item in user_items:
                # Convert timestamp to datetime
                last_viewed = item.lastViewed
                if last_viewed == -1:
                    last_viewed = datetime.utcfromtimestamp(0)  # Set to epoch time
                else:
                    last_viewed = datetime.fromtimestamp(last_viewed / 1000)  # Convert milliseconds to seconds

                # Convert datetime to ArcGIS Online format date (e.g., "yyyy-mm-ddTHH:mm:ssZ")
                last_viewed_agol_format = last_viewed.strftime("%Y-%m-%d")

                if last_viewed.date() <= self.target_date:
                    # Calculate credits per month based on item type and size
                    item_type = item.type  # Assuming the 'type' attribute is present in the item object
                    item_size = item.size / (1024 * 1024)  # Convert bytes to megabytes
                    credits_per_month = self._calculate_credits_per_month(item_type, item_size)

                    item_info = {
                        "item_name": item.title,
                        "item_id": item.itemid,
                        "item_url": item.url,
                        "last_viewed_date": last_viewed_agol_format,
                        "username": user.username,
                        "full_name": user.fullName,
                        "email": user.email,
                        "item_size_mb": item_size,
                        "item_type": item_type,
                        "estimated_credits_per_month": credits_per_month,
                        "estimated_cost_per_month": credits_per_month * 0.1
                    }
                    all_items.append(item_info)

        # Convert the list of items to a pandas DataFrame
        items_df = pd.DataFrame(all_items)

        return items_df

    def get_sorted_user_details(self):
        # Use self.gis to access the GIS instance
        gis = self.gis
        items_before_date_all_users_df = self.get_items_before_date_all_users()
        # Extract unique usernames
        unique_usernames = items_before_date_all_users_df['username'].unique()

        # Group the DataFrame by 'username' and calculate the total counts of items owned by each user
        user_item_counts = items_before_date_all_users_df.groupby('username')['item_id'].count()

        # Group the DataFrame by 'username' and sum the estimated credits per month, item size, and estimated cost per month for each user
        user_aggregated_data = items_before_date_all_users_df.groupby('username').agg({
            'estimated_credits_per_month': 'sum',
            'item_size_mb': 'sum',
            'estimated_cost_per_month': 'sum'
        })

        # Create a list to store user details including last login date, full name, email, item counts, total credit cost, total item size, and total cost per month
        user_details_list = []

        # Iterate through unique usernames and query the last login date, full name, email address, item counts, total credit cost, total item size, and total cost per month for each user
        for username in unique_usernames:
            user = gis.users.get(username)
            last_login_date_str = (datetime.fromtimestamp(user.lastLogin / 1000)).strftime("%Y-%m-%d")
            item_count = user_item_counts.get(username, 0)
            total_credits = user_aggregated_data.loc[username, 'estimated_credits_per_month']
            total_item_size_mb = user_aggregated_data.loc[username, 'item_size_mb']
            total_cost_per_month = user_aggregated_data.loc[username, 'estimated_cost_per_month']
            user_details = {
                "username": username,
                "last_login_date": last_login_date_str,
                "full_name": user.fullName,
                "email": user.email,
                "item_count": item_count,
                "total_credits": total_credits,
                "total_item_size_mb": total_item_size_mb,
                "total_cost_per_month": total_cost_per_month
            }
            user_details_list.append(user_details)

        # Sort the user details list by highest total cost per month
        sorted_user_details = sorted(user_details_list, key=lambda x: x['total_cost_per_month'], reverse=True)

        return pd.DataFrame(sorted_user_details)
    


In [6]:
# Create an instance of the ArcGISDataAnalyzer class with target date "2023-01-01"
analyzer = ArcGISDataAnalyzer("2023-01-01")

# Call the get_items_before_date_all_users() method on the analyzer instance
items_df = analyzer.get_items_before_date_all_users()

items_df.head()

,item_name,item_id,item_url,last_viewed_date,username,full_name,email,item_size_mb,item_type,estimated_credits_per_month,estimated_cost_per_month
0,UNGEGN WGN Dashboard Test,eec0eff6af864b14b7e4d1db1668acf7,None,1970-01-01,abenjamin_esriaiddev,Alice Benjamin,abenjamin@esri.com,0.139005,Web Experience,0.033361,0.003336
1,CBC Rare Bird Observation Editor,8832465d17a742b4bb1e3aaf7556ad1c,//esriaiddev.maps.arcgis.com/sharing/rest/cont...,1970-01-01,AGiron_aid,Amanda Giron,AGiron@esri.com,0.000000,Code Attachment,0.000000,0.000000
2,Christmas Bird Count Rare Bird Reviewer Experi...,538febabfcc84ac2bd82a6606e21ee8c,https://experience.arcgis.com/experience/538fe...,2022-03-22,AGiron_aid,Amanda Giron,AGiron@esri.com,0.012760,Web Experience,0.003062,0.000306
3,Christmas Bird Count Rare Bird Reviewer Map,8998185999714d7dbdd36517c9516567,,2022-04-07,AGiron_aid,Amanda Giron,AGiron@esri.com,0.019292,Web Map,0.004630,0.000463
4,Christmas Bird Count Rare Bird Attachment View...,279f32030673429981a1b13bfe255721,https://esriaiddev.maps.arcgis.com/apps/instan...,2022-04-07,AGiron_aid,Amanda Giron,AGiron@esri.com,0.000591,Web Mapping Application,0.000142,0.000014


In [8]:
total_estimated_cost_per_month = items_df['estimated_cost_per_month'].sum()
total_estimated_cost_per_month

190.25062300872804

In [9]:
df2 = analyzer.get_sorted_user_details()
df2

,username,last_login_date,full_name,email,item_count,total_credits,total_item_size_mb,total_cost_per_month
0,retiredcontent_esriaiddev,2024-06-28,Rami Alouta,ralouta@esri.com,47,863.509393,3597.955804,86.350939
1,ralouta.aiddev,2025-02-19,Rami Alouta,ralouta@esri.com,32,731.038907,3045.995446,73.103891
2,ckwon_aid,2025-02-11,Calvin Kwon,ckwon@esri.com,63,107.146538,446.443910,10.714654
3,dgadsden_aid2,2022-12-01,David Gadsden,dgadsden@esri.com,26,56.028401,233.451673,5.602840
4,vdesrosier_esriaiddev,2025-02-18,Valerie Desrosier,vdesrosier@esri.com,25,48.229251,200.955213,4.822925
5,apfister_EsriAidDev,2025-02-19,Adam Pfister,apfister@esri.com,52,26.755405,111.480854,2.675540
6,ltroup_EsriAidDev,2025-02-19,Lydia Troup,ltroup@esri.com,13,18.924189,78.850788,1.892419
7,ssawaya_aid,2025-02-15,Salim Sawaya,ssawaya@esri.com,28,17.559741,73.165587,1.755974
8,cbrigham_aid,2024-01-08,Charles Brigham,cbrigham@esri.com,45,7.996989,33.320788,0.799699
9,cterborgh_aid,2025-02-14,Carmelle Terborgh,cterborgh@esri.com,41,7.030860,29.295251,0.703086
